In [1]:
import networkx as nx
import matplotlib as plt
import numpy as np
import pandas as pd
from itertools import permutations

In [2]:
def randomWave(wavelist):
    random_seed=np.random.randint(len(wavelist))
    return wavelist[random_seed]

In [3]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges      

def pathBand(path):
    path_len=len(path)
    #print(path)
    #print(path[0][0])
    resource=Resource[path[0][0]][path[0][1]][path[0][2]]
    for i in range(1,path_len):
        resource=list(map(lambda x: x[0]+x[1], zip(resource, Resource[path[i][0]][path[i][1]][path[i][2]])))
    pathbandList=zero_runs(resource) 
    
    return pathbandList

In [35]:
def bypassType(path):
    path_len=len(path)
    if(path_len==1):
        bypass_type=0
    elif(path_len==2):
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=1
        else:
            bypass_type=0
            
    elif(path_len==3):       
        edgeLoc0=edges_list.index((path[0][0],path[0][1]))
        edgeLoc1=edges_list.index((path[1][0],path[1][1]))
        edgeLoc2=edges_list.index((path[2][0],path[2][1]))
        if(path[1] in Out_bypassEdges[edgeLoc0][path[0][2]] and path[0] in In_bypassEdges[edgeLoc1][path[1][2]] ) :
            bypass_type=1
        elif(path[2] in Out_bypassEdges[edgeLoc0][path[1][2]] and path[1] in In_bypassEdges[edgeLoc1][path[2][2]] ):
            bypass_type=1
        else:
            bypass_type=0
                       
            
    return bypass_type
                
            

In [5]:
def pathBand_Ocuupy(loc1,loc2,wave1,band,index):
    path=SD_pathLink[loc1][loc2]
    pathBand=ALL_SD_bandwidth[loc1][loc2]
    for i in range(len(path)):
        for j in range(band):
            #print(path)
            #print(wave1)
            #print(Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j])
            Resource[path[i][0]][path[i][1]][path[i][2]][wave1+j]+=index+1
            
    return

        
def pathUpdate():
    SR_bandwidth=[]
    #MaxBand=[]
    Hops=[]
    for i in range(len(SD_pathLink)):
        band=[]
        #Max=[]
        hops=[]        
        for j in range(len(SD_pathLink[i])):
            #cost.append(pathCost(SD_pathLink[i][j]))
            hops.append(len(SD_pathLink[i][j]))
            b=pathBand(SD_pathLink[i][j])
            #Max.append(max(b[:,1] - b[:,0]))
            band.append(b)            
        #SR_pathLink_cost.append(cost)
        SR_bandwidth.append(band)
        #MaxBand.append(Max)
        Hops.append(hops)
    global ALL_SD_bandwidth
    ALL_SD_bandwidth=SR_bandwidth
        
    return

In [6]:
def CoreSetup():
    Nodes_list=list(G.nodes)
    #Link_bypass=[]
    for i in Nodes_list:
        neighborList=list(G.neighbors(i))
        #bypasscore_num=len(neighborList)-1# set the bypass core number within each fiber 
        for j in range(len(neighborList)):
            dis=1
            while(j+dis<len(neighborList)):
                input_loc=edges_list.index((neighborList[j],i))
                output_loc=edges_list.index((i,neighborList[j+dis]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j+dis],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                input_loc=edges_list.index((neighborList[j+dis],i))
                output_loc=edges_list.index((i,neighborList[j]))
                successFlag=0
                for m in range(7):
                    if(len(bypassEdges[input_loc][m])==0):
                        for n in range(7):
                            if(len(bypassEdges[output_loc][n])==0):
                                bypassEdges[output_loc][n].append(((neighborList[j+dis],i,m)))
                                bypassEdges[input_loc][m].append(((i,neighborList[j],n)))
                                successFlag=1
                                break
                        if(successFlag==1):
                            break
                                                                                                        
                dis+=1
    return;

In [7]:
def edgeToLinks(path):
    pathlist=[]
    path_len=len(path)
    corePossib=7**path_len
    for i in range(corePossib):
        List=[]
        List.append((path[0]+(i%7,)))
        edgeLoc=edges_list.index(path[0])
        
        if(len(In_bypassEdges[edgeLoc][i%7])>0):
            List=[]
            continue
        for j in range(1,path_len):
            List.append((path[j]+((int(i/(7**j)%7)),)))
            edgeLoc0=edges_list.index(path[j-1])
            edgeLoc1=edges_list.index(path[j])
            if(len(Out_bypassEdges[edgeLoc0][List[j-1][2]])>0 and List[j] not in Out_bypassEdges[edgeLoc0][List[j-1][2]] ) :

                List=[]
                break
            if(len(In_bypassEdges[edgeLoc1][List[j][2]])>0 and List[j-1] not in In_bypassEdges[edgeLoc1][List[j][2]] ) :

                List=[]
                break
        edgeLoc=edges_list.index(path[path_len-1])
        if(len(Out_bypassEdges[edgeLoc][int(i/(7**(path_len-1)))])>0):            
            List=[]
            continue 
        if(len(List)>0):
            pathlist.append(List)    
    return pathlist

In [8]:
def PathAlgorithm_1(PathList,hopsList,bandwidth,pathBandList,bypassList,wave,list1,list2):
    Max=np.zeros(len(pathBandList))
    Sum=np.zeros(len(pathBandList))
    for i in range(len(pathBandList)):
        #print (pathBandList[i][:,1] - pathBandList[i][:,0])
        #print(i)
        #print(PathList[0])
        Max[i]=max(pathBandList[i][:,1] - pathBandList[i][:,0])
        Sum[i]=sum(pathBandList[i][:,1] - pathBandList[i][:,0])
    
    infor={"place":range(len(PathList)),
           "bypass":bypassList,
           "bandwidth": Max,
           "hops": hopsList,
           "sum":Sum
         }
    
    Path_infor=pd.DataFrame(infor)     
    #print(Path_infor)
    df=Path_infor.sort_values(by=list1,ascending=list2)  
    df.reset_index(drop=True, inplace=True)#reset the index sequence
    #print(df)
    Path=[]
    successful_flag=0
    for i in range(len(df)):
        if(df["bandwidth"][i]>bandwidth):
            for j in pathBandList[df["place"][i]]:
                if(wave>=j[0] and wave+bandwidth<=j[1]):
                    Path=PathList[df["place"][i]]
                    traffic_feedback.append(df["bypass"][i])
                    successful_flag=1
                    break 
        if(successful_flag==1):
            break
    if(len(Path)==0):
        traffic_feedback.append(-1)
        #print(df)
    return Path

In [9]:
def PathOccupy_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((percent))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((percent))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((percent))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [10]:
def Link_percentage():
    Edge=[]
    Link=[]
    for i in edges_list:
        Edge.append(str(i))
    for j in range(7):        
        occupy=[]
        for i in edges_list:
            #print(i)
            #print(list(np.nonzero(Resource[i[0]][i[1]][j])))
            k=np.array(Resource[i[0]][i[1]][j])
            per=np.count_nonzero(k)/160
            occupy.append(per)
        Link.append(occupy)
    df=pd.DataFrame(Link,columns=Edge)
    #print(df)
    #df.rename()
    return df
        

In [11]:
def Path_percent():
    Bypass_2=[]
    Bypass_1=[]
    Bypass_0=[]
    for i in range(len(edges_list)):
        if(edges_list[i] in Traffic_SD_pairs):
            for j in range(len((SD_pathLink[i]))):
                #percent=1-(sum(ALL_SD_bandwidth[i][j][:,1]-ALL_SD_bandwidth[i][j][:,0])-1)/160
                if(Bypass_Type[i][j]==2):
                    Bypass_2.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==1):
                    Bypass_1.append((SD_pathLink[i][j]))
                elif(Bypass_Type[i][j]==0):
                    Bypass_0.append((SD_pathLink[i][j]))
    Bypass=[]
    Bypass.append(Bypass_0)
    Bypass.append(Bypass_1)
    
    Bypass.append(Bypass_2)
    
    return Bypass

In [12]:
TrafficCost=0
node_list=[0,1,2,3]
Traffic_SD_pairs=[(0,3),(0,1),(0,2),(1,2),(2,3),(1,3),(2,1)]
SR_pair=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(0,3),(3,0),(1,3),(3,1)]
edges_list=[(0,1),(1,0),(1,2),(2,1),(0,2),(2,0),(2,3),(3,2),(1,3),(3,1)]
bypassEdges=[]
In_bypassEdges=[]
Out_bypassEdges=[]
for i in range(len(edges_list)):
    BY=[]
    BY0=[]
    BY1=[]
    
    for j in range(7):
        BY.append([])
        BY0.append([])
        BY1.append([])
    bypassEdges.append(BY)
    In_bypassEdges.append(BY0)
    Out_bypassEdges.append(BY1)
    

   
#先对D=2的节点进行bypass设置，在对D=3的节点进行bypass设置
#每次设置都在该link中不存在

G=nx.DiGraph()
G.add_nodes_from(node_list)
G.add_edges_from(edges_list)
nx.draw(G)

#plt.show()


link_list=[] 

for k in range(len(edges_list)):
    link_list0=[]
    for i in range(7):
        link_list0.append(edges_list[k]+(i,)) 
    link_list.append(link_list0)

SD_path=[]
SR_pathLink=[]
for i in range(len(SR_pair)):
    path0=[]
    for path in nx.all_simple_paths(G, source=SR_pair[i][0], target=SR_pair[i][1]):
        path_edges=[]            
        for j in range(len(path)-1):
            path_edges.append((path[j],path[j+1]))
        path0.append(path_edges)
    SD_path.append(path0)


CoreSetup() 
for i in range(len(bypassEdges)):
    Link=bypassEdges[i]
    for j in range(7):
        if(len(Link[j])>0):
            if(Link[j][0][0]==edges_list[i][1]):
                
                Out_bypassEdges[i][j]=Link[j]
            elif(Link[j][0][1]==edges_list[i][0]):
                In_bypassEdges[i][j]=Link[j]
                

In [13]:
SD_pathLink=[]  
for i in range(len(SD_path)):
    Links=[]
    for j in range(len(SD_path[i])):
        Links=Links+edgeToLinks(SD_path[i][j])
        
    SD_pathLink.append(Links)


In [14]:
Tag_range = list(permutations(range(3), 3))
Tag_list=["hops","bypass","sum"]
FalseorTrue=[]
for i in range(8):
    k1=int(i%2)
    k2=int((i%4)/2)
    k3=int(i/4)
    FalseorTrue.append((k1,k2,k3))
    
print(FalseorTrue)

FalseTrueList=[False,True]


[(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0), (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)]


In [144]:
Success_list=[]
Tag_List=[]
True_List=[]
for tag_selection in range(1):
    for true_selection in range(1):
    
    
        #Tag=Tag_range[tag_selection]
        #choose=FalseorTrue[true_selection]
        list1=["hops","bypass","sum"]
        list2=[True,False,True]
        #for i in range(len(Tag)):
            #list1.append(Tag_list[Tag[i]])
        #for i in range(len(choose)):
            #list2.append(FalseTrueList[choose[i]])
        
        Total_bandwidth=161
        Resource=np.zeros([4,4,7,161]).tolist()
        ALL_SD_bandwidth=[]
        MaxBand=[]
        Hops=[]
        Bypass_Type=[]
        for i in range(len(SD_pathLink)):
            Band=[]
            Max=[]
            Hhops=[]
            Bypass=[]
            for j in range(len(SD_pathLink[i])):
                Hhops.append(len(SD_pathLink[i][j]))
                b=pathBand(SD_pathLink[i][j])
                Max.append(max(b[:,1] - b[:,0]))
                Band.append(b)
                Bypass.append(bypassType(SD_pathLink[i][j]))
            ALL_SD_bandwidth.append(Band)
            MaxBand.append(Max)
            Hops.append(Hhops)
            Bypass_Type.append(Bypass)
        

        
        
        # In[214]:
        
        
        Traffic=pd.read_csv('traffic.csv',sep=',',names=['S','D','band'])
        SuccessFlag=np.zeros(len(Traffic))
        BypassFlag=np.zeros(len(Traffic))
        traffic_wave=[]
        traffic_path=[]
        traffic_feedback=[]
        Path_band_usage=[]
        Traffic_num=len(Traffic)
        for i in range(Traffic_num):
            traffic=(Traffic['S'][i],Traffic['D'][i])
            band=Traffic['band'][i]
            traffic_Path=SD_pathLink[SR_pair.index(traffic)]
            traffic_Hops=Hops[SR_pair.index(traffic)]
            traffic_Band=ALL_SD_bandwidth[SR_pair.index(traffic)]
            traffic_Bypass=Bypass_Type[SR_pair.index(traffic)]
            wave=list(range(Total_bandwidth-1))
            while(len(wave)):
                PathFlag=0
                select_wave=randomWave(wave)
                print(select_wave)
                selectPath=PathAlgorithm_1(traffic_Path,traffic_Hops,band,traffic_Band,traffic_Bypass,select_wave,list1,list2)
                if(len(selectPath) and select_wave+band<=160):
                    Pathloc=traffic_Path.index(selectPath)
                    pathBand_Ocuupy(SR_pair.index(traffic),Pathloc,select_wave,band,i)
                    PathFlag=1
                    traffic_wave.append(select_wave)
                    traffic_path.append(selectPath)
                    pathUpdate()
                    if(PathFlag==1):
                        SuccessFlag[i]=1
                    break
                else:
                    wave.remove(select_wave)

            
                
        Success_list.append(SuccessFlag.sum()) 
        Tag_List.append(list1)
        True_List.append(list2)

62
19
115
45
120
133
94
56
54
109
19
115
96
67
122
93
131
87
157
27
71
29
135
9
41
129
36
148
46
21
89
117
94
19
140
94
146
19
139
13
40
56
11
116
132
130
141
136
24
0
54
11
134
153
5
2
85
16
88
144
64
136
142
42
151
3
134
149
143
21
23
116
156
40
111
68
74
151
14
56
5
51
64
89
120
121
10
150
151
91
9
78
104
152
68
112
51
157
84
83
144
103
63
96
127
28
108
8
72
126
38
119
87
138
69
3
23
100
38
137
66
127
81
136
60
114
40
66
123
103
124
31
16
115
124
48
72
90
22
40
89
118
71
13
95
62
71
77
149
33
132
39
58
130
101
122
117
83
73
52
73
56
22
16
67
63
129
84
115
47
135
21
145
149
11
61
19
90
113
93
31
61
131
145
66
122
113
151
103
137
143
80
98
88
153
21
32
11
59
56
79
131
150
9
69
5
132
19
20
104
42
2
35
49
26
154
156
76
14
3
142
157
93
150
126
34
55
125
62
93
101
122
34
131
99
129
33
61
129
130
124
62
104
33
55
65
70
145
2
6
116
50
41
80
68
116
36
36
44
111
130
45
21
36
158
91
52
22
57
35
109
39
35
140
22
35
155
63
95
51
53
88
96
24
79
148
107
154
34
23
25
125
63
128
10
41
150
43
138
122

91
22
100
115
93
139
122
89
83
5
148
73
139
36
64
56
84
9
51
126
27
109
75
107
99
6
35
8
36
145
130
49
71
114
73
156
34
154
22
112
24
111
48
66
107
22
54
134
121
155
24
132
89
89
146
9
155
90
31
126
10
75
88
30
104
32
6
76
42
122
59
37
144
76
93
20
134
76
11
104
126
102
97
104
19
132
94
137
114
14
60
57
112
155
13
149
9
91
86
44
19
12
132
72
116
81
87
16
89
101
60
108
13
42
56
27
115
122
5
122
132
82
115
49
29
126
73
127
134
19
155
114
129
43
93
129
56
57
37
48
62
13
137
52
102
52
83
14
23
34
158
112
140
125
57
64
28
127
47
77
8
120
137
58
34
96
109
82
144
14
51
124
40
136
54
126
0
48
50
99
11
136
144
76
113
89
128
49
69
136
133
91
114
83
9
58
33
107
16
51
40
110
96
84
159
1
43
9
52
128
42
9
129
51
75
71
146
39
90
90
140
52
51
85
106
140
9
140
114
37
75
148
100
26
58
70
148
70
70
23
61
89
74
152
145
154
122
35
69
3
29
36
2
53
146
59
140
106
26
76
135
103
128
75
148
127
13
54
152
44
117
73
65
128
44
133
13
131
133
127
19
74
95
125
73
79
17
129
21
70
106
143
37
125
50
118
114
91
141
117


124
157
86
62
58
16
84
32
64
103
9
56
90
106
134
156
7
80
130
108
97
137
141
121
138
28
48
39
45
159
20
125
87
25
142
111
0
143
126
38
105
114
147
132
113
122
66
79
31
15
17
115
3
74
133
42
26
34
92
112
52
139
81
77
117
140
155
35
54
102
82
110
19
44
127
68
89
69
8
153
61
33
116
47
59
158
144
154
70
65
22
136
4
18
88
27
83
14
12
49
123
10
43
99
96
37
46
149
72
23
55
145
135
73
131
13
118
120
95
128
57
2
150
21
100
129
29
119
6
42
107
136
82
55
145
93
152
91
14
8
107
49
44
127
37
86
109
147
140
16
141
85
88
118
87
21
71
110
44
137
156
116
136
7
74
97
157
108
104
86
129
3
152
107
124
61
155
139
106
39
32
45
34
88
2
60
36
131
30
126
134
101
150
127
6
31
67
50
53
140
149
112
147
64
8
48
148
120
24
18
12
38
17
158
130
141
20
113
11
70
69
4
9
55
73
159
49
115
25
57
54
133
92
100
47
75
63
81
27
84
121
65
151
82
111
13
153
68
80
85
51
95
1
72
10
62
15
52
83
66
79
5
105
96
146
89
0
19
26
29
56
91
28
78
98
43
135
143
118
154
16
33
46
58
122
42
125
102
90
94
99
35
109
145
128
103
119
41
77
132
14

In [145]:
print(SuccessFlag)

[1. 1. 1. ... 0. 1. 1.]


In [146]:
print(Tag_List[Success_list.index(max(Success_list))])
print(True_List[Success_list.index(max(Success_list))])

['hops', 'bypass', 'sum']
[True, False, True]


In [147]:
print(type(Resource))

<class 'list'>


In [148]:
print(Link_percentage())

    (0, 1)   (1, 0)   (1, 2)   (2, 1)   (0, 2)   (2, 0)   (2, 3)   (3, 2)  \
0  0.70625  0.68750  0.53750  0.68750  0.68750  0.70625  0.74375  0.51875   
1  0.53750  0.68750  0.53750  0.74375  0.62500  0.70000  0.76875  0.87500   
2  0.82500  0.73125  0.70000  0.62500  0.76875  0.87500  0.78125  0.57500   
3  0.82500  0.83125  0.74375  0.51875  0.82500  0.81875  0.88750  0.85625   
4  0.74375  0.81875  0.86875  0.78125  0.78125  0.81250  0.78750  0.82500   
5  0.70000  0.80000  0.80000  0.86875  0.68750  0.68125  0.77500  0.71875   
6  0.68125  0.81250  0.77500  0.74375  0.65625  0.75000  0.85000  0.69375   

    (1, 3)   (3, 1)  
0  0.82500  0.73125  
1  0.74375  0.53750  
2  0.57500  0.78125  
3  0.75625  0.87500  
4  0.86250  0.82500  
5  0.77500  0.78750  
6  0.81250  0.69375  


In [149]:
print(Success_list)

[2982.0]


In [150]:
k=0
for i in traffic_path:
    k+=len(i)
print(k/len(traffic_path))

1.3571428571428572


In [151]:
occupy=0
for j in range(7):        
    
    for i in edges_list:
            #print(i)
            #print(list(np.nonzero(Resource[i[0]][i[1]][j])))
        k=np.array(Resource[i[0]][i[1]][j])
        S=np.count_nonzero(k)
        occupy+=S
print(occupy)
print(occupy/(7*10))

8309
118.7


In [152]:
k=0
for i in range(Traffic_num):
    if(SuccessFlag[i]==1):
        k+=Traffic['band'][i]
print(k)

5887
